In [ ]:
import tiledb
import xarray as xr
import numpy as np

# TileDB Backend for xarray

This example shows how to use the TileDB backend for xarray. This example requires `tiledb-cf` to be installed and uses the `tiledb`, `xarray`, and `numpy` libraries. 


## Example 1. Opening a dense array

The TileDB-xarray backend supports opening dense arrays in xarray. Integer TileDB dimensions that have a domain that starts with `0` are treated as NetCDF-like dimensions. Dimensions that start at a different value or have a non-integer domain are treated like NetCDF coordinates.

In this example, we create an array with the following properties:

Dimensions:

| Name | Domain   | Data Type |
|:----:|:---------|:----------|
| x    | (0, 99)  | uint64    |
| y    | (0, 149) | uint64    |
| t    | (1, 36)  | uint64    |

Attributes:

| Name    | Data Type | Details                        |
|:-------:|:----------|:-------------------------------|
| ripple1 | float64   | sin(t * (x^2 + y^2)) / (t + 1) |
| riplle2 | float64   | cos(t * (x^2 + y^2)) / (t + 1) |

Here, xarray will open `x` and `y` as dimensions, `t` as a coordinate, and `ripple1` and `ripple2` as variables.

To assign xarray attributes (metadata) to variables and coordinates we use the prefix `__tiledb_attr.{attr_name}.` or `__tiledb_dim.{dim_name}` before the TileDB metadata keyword.

In [ ]:
uri1 = "output/xarray-example-1"
if not tiledb.object_type(uri1):
    x_size = 100
    y_size = 250
    t_size = 36
    schema = tiledb.ArraySchema(
        domain=tiledb.Domain(
            tiledb.Dim("x", domain=(0, x_size - 1), dtype=np.uint64),
            tiledb.Dim("y", domain=(0, y_size - 1), dtype=np.uint64),
            tiledb.Dim("time", domain=(1, t_size), dtype=np.uint64),
        ),
        attrs=(
            tiledb.Attr("ripple1", np.float64),
            tiledb.Attr("ripple2", np.float64),
        ),
    )
    tiledb.Array.create(uri1, schema)
    with tiledb.open(uri1, mode="w") as array:
        array[:, :, :] = {
            "ripple1": np.fromfunction(
                lambda x,y,t: np.sin(t * (x ** 2 + y **2 )) / (t + 1), 
                (x_size, y_size, t_size)
            ),
            "ripple2": np.fromfunction(
                lambda x,y,t: np.cos(t * (x ** 2 + y **2 )) / (t + 1), 
                (x_size, y_size, t_size)
            ),
        }
        array.meta["__tiledb_attr.ripple1.description"] = "sin(t * (x^2 + y^2)) / (t + 1)"
        array.meta["__tiledb_attr.ripple2.description"] = "cos(t * (x^2 + y^2)) / (t + 1)"
        array.meta["__tiledb_dim.time.description"] = "time in seconds"
        array.meta["description"] = "Small example dense array"
else:
    print(f"A TileDB {tiledb.object_type(uri1)} already exists at '{uri1}'")

The TileDB array is opened with xarray using the `tiledb` engine. This allows for xarray to access the data using it's standard lazy-loading. Once we've created the dataspace we can access and slice the data using standard xarray capabilities.

In [ ]:
ds = xr.open_dataset(uri1, engine="tiledb")
ds

## Example 2. Handling Coordinates

In TileDB, an attribute and a dimension in the same array cannot have the same name. In order to handle "coordinates" (a xarray variable and dimension with the same name), the suffix `.data` and `.index` will be stripped from the name of TileDB attributes and dimensions.

In this example, we show creating a coordinate for a one-dimension TileDB array.

In [ ]:
uri2 = "output/xarray-example-2"
if not tiledb.object_type(uri2):
    schema = tiledb.ArraySchema(
        domain=tiledb.Domain(tiledb.Dim("x", domain=(0, 63), dtype=np.int64)),
        attrs=[
            tiledb.Attr("x.data", np.float64),
            tiledb.Attr("y", np.float64),
        ]
    )
    tiledb.Array.create(uri2, schema)
    x_values = np.linspace(-1.0, 1.0, 64)
    with tiledb.open(uri2, mode="w") as array:
        array[:] = {
            "x.data": x_values,
            "y": np.exp(- x_values / 2.0)
        }
else:
    print(f"A TileDB {tiledb.object_type(uri2)} already exists at '{uri2}'")

In [ ]:
ds2 = xr.open_dataset(uri2, engine="tiledb")
ds2

In [ ]:
ds2.plot.scatter(x="x", y="y")